In [1]:
import glob
import os
import pandas as pd
import numpy as np

In [2]:
# Assumes jupyter started in 
df_regions = pd.read_csv('../localrankdata_2_23_22.csv')

In [3]:
df_regions

,Unnamed: 0,X,Country,StartYear,EndYear,PopulationAt2015,CountryCode,Continent,Subregion,Allfreq,...,USAmean_v,IMPsd,IMPfreq,IMPmean,IMPfreq_n,IMPse,IMPmean_n,IMPfreq_v,IMPse_v,IMPmean_v
0,1,1,"Afghanistan, Islamic Republic of",2015,2015,34413603,AFG,Asia,Southern Asia,86560,...,0.724901,6.535984,4759,-0.321872,0.053812,0.095766,3.591542,0.158755,0.028537,0.783829
1,2,2,"Afghanistan, Islamic Republic of",2015,2021,34413603,AFG,Asia,Southern Asia,592498,...,0.740164,7.122214,46046,-0.170888,0.076150,0.033530,3.133380,0.224656,0.009992,0.731660
2,3,3,"Afghanistan, Islamic Republic of",2016,2016,34413603,AFG,Asia,Southern Asia,99965,...,0.774720,6.935615,6908,-0.554256,0.068394,0.083879,3.549607,0.201773,0.024995,0.779054
3,4,4,"Afghanistan, Islamic Republic of",2017,2017,34413603,AFG,Asia,Southern Asia,96514,...,0.732995,7.085932,7203,-0.391509,0.073772,0.083976,3.267358,0.217638,0.025024,0.746915
4,5,5,"Afghanistan, Islamic Republic of",2018,2018,34413603,AFG,Asia,Southern Asia,121433,...,0.725754,7.976021,9071,-0.185854,0.073983,0.084149,2.749676,0.218262,0.025076,0.687969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
747,748,748,"Zimbabwe, Republic of",2017,2017,13814629,ZWE,Africa,Eastern Africa,209712,...,0.451754,7.548791,6356,-0.968585,0.030146,0.094940,1.087782,0.088936,0.028291,0.498733
748,749,749,"Zimbabwe, Republic of",2018,2018,13814629,ZWE,Africa,Eastern Africa,225702,...,0.555700,6.552132,7592,-0.151263,0.033301,0.075577,1.503617,0.098242,0.022521,0.546083
749,750,750,"Zimbabwe, Republic of",2019,2019,13814629,ZWE,Africa,Eastern Africa,182888,...,0.488202,6.299169,7616,-1.068543,0.041561,0.072252,1.438266,0.122611,0.021530,0.538642
750,751,751,"Zimbabwe, Republic of",2020,2020,13814629,ZWE,Africa,Eastern Africa,101751,...,0.510544,6.707062,4349,-1.406127,0.042329,0.102199,1.052292,0.124877,0.030454,0.494692


In [4]:
continents = df_regions['Continent'].unique()

In [5]:
continents

array(['Asia', 'Africa', 'Europe'], dtype=object)

In [6]:
subregions = df_regions['Subregion'].unique()

In [7]:
subregions

array(['Southern Asia', 'Northern Africa', 'Middle Africa',
       'Western Asia', 'Western Africa', 'Southern Africa',
       'South-eastern Asia', 'Eastern Africa', 'Eastern Asia',
       'Central Asia'], dtype=object)

End goal table
-----

- Columns consist of the **location**, which can be at the resolution of *continents* (Asia, Africa) or the resolution of *subregions* (Southern Asia, Western Asia, etc.). 
- Rows consist of great power **actor** (China, US, Russia, France, India).
- Entries in the table consist of the *sum*, *mean*, and *std. dev.* of the # of articles produced by each country's local press in the **location** of interest (continent or subregion).

Compute the country membership of the **locations** (continent and subregion)

In [8]:
continent_country = {}
subregion_country = {}

In [9]:
for i, row in df_regions.iterrows():
    if row['Continent'] != 'Europe':
        if continent_country.get(row['Continent']) is None:
            continent_country[row['Continent']] = set([row['Country']])
        else:
            continent_country[row['Continent']].add(row['Country'])
            
    if row['Subregion'] == 'Eastern Asia':
        continue
    if subregion_country.get(row['Subregion']) is None:
        subregion_country[row['Subregion']] = set([row['Country']])
    else:
        subregion_country[row['Subregion']].add(row['Country'])

In [10]:
countriesc = set()
for _, countries in continent_country.items():
    for c in countries:
        countriesc.add(c)

In [11]:
countriess = set()
for _, countries in subregion_country.items():
    for c in countries:
        countriess.add(c)

In [12]:
countriess-countriesc

{'Russian Federation'}

In [13]:
'Russian Federation' in countriesc

False

In [14]:
continent_country['Asia'].add('Russian Federation')

In [15]:
subregion_country['Central Asia']

{'Kazakhstan, Republic of',
 'Kyrgyz Republic',
 'Russian Federation',
 'Tajikistan, Republic of',
 'Turkmenistan',
 'Uzbekistan, Republic of'}

Changes
- Mauritania: Western -> Northern
- Sudan: Northern -> Eastern
- Angola: Middle/Central -> Southern
- Mozambique, Zambia, Zimbabwe, Malawi: Eastern -> Southern

In [16]:
subregion_country['Western Africa'].remove('Mauritania, Islamic Republic of')
assert not 'Mauritania, Islamic Republic of' in subregion_country['Western Africa']
subregion_country['Northern Africa'].add('Mauritania, Islamic Republic of')
assert 'Mauritania, Islamic Republic of' in subregion_country['Northern Africa']

In [17]:
subregion_country['Northern Africa'].remove('Sudan, Republic of')
assert not 'Sudan, Republic of' in subregion_country['Northern Africa']
subregion_country['Eastern Africa'].add('Sudan, Republic of')
assert 'Sudan, Republic of' in subregion_country['Eastern Africa']

In [18]:
subregion_country['Middle Africa'].remove('Angola, Republic of')
assert not 'Angola, Republic of' in subregion_country['Middle Africa']
subregion_country['Southern Africa'].add('Angola, Republic of')
assert 'Angola, Republic of' in subregion_country['Southern Africa']

In [19]:
subregion_country['Eastern Africa'].remove('Mozambique, Republic of')
subregion_country['Eastern Africa'].remove('Zambia, Republic of')
subregion_country['Eastern Africa'].remove('Zimbabwe, Republic of')
subregion_country['Eastern Africa'].remove('Malawi, Republic of')

assert not 'Mozambique, Republic of' in subregion_country['Eastern Africa']
assert not 'Zambia, Republic of' in subregion_country['Eastern Africa']
assert not 'Zimbabwe, Republic of' in subregion_country['Eastern Africa']
assert not 'Malawi, Republic of' in subregion_country['Eastern Africa']

subregion_country['Southern Africa'].add('Mozambique, Republic of')
subregion_country['Southern Africa'].add('Zambia, Republic of')
subregion_country['Southern Africa'].add('Zimbabwe, Republic of')
subregion_country['Southern Africa'].add('Malawi, Republic of')

assert 'Mozambique, Republic of' in subregion_country['Southern Africa']
assert 'Zambia, Republic of' in subregion_country['Southern Africa']
assert 'Zimbabwe, Republic of' in subregion_country['Southern Africa']
assert 'Malawi, Republic of' in subregion_country['Southern Africa']

For each location and each great power actor, compute the total number of local articles produced.

In [20]:
actors = ['CHN', 'USA', 'RUS', 'FRA', 'IND']

In [21]:
subregion_actor_stat = {}

In [22]:
actor_pos = {
    'USA': 0,
    'IND': 1,
    'RUS': 2,
    'FRA': 3,
    'CHN': 4,
    'ALL': 5
}

In [23]:
for actor in actors:
    total_cnts = []
    for subregion, country in subregion_country.items():
        # compute stats
        cnts = []
        for c in country:
            if (c == 'Russian Federation' and actor == 'RUS') or (c == 'India, Republic of' and actor == 'IND'):
                continue
                
            # fetch local report counts for this actor
            p = 'CountrySourceCounts'+ '/' + c + '/' + 'local_stats_2015-02-18_2021-03-08_' + actor + '.csv'
            if (os.path.isfile(p)):
                df_cnt = pd.read_csv(p)
                cnt = df_cnt['NumArticles'].values[0]
                cnts.append(cnt)
                total_cnts.append(cnt)
            
        total = sum(cnts)
        mean = int(np.mean(cnts))
        std = int(np.std(cnts))
        
        stat = (total, mean, std)
        
        subregion_actor_stat[(subregion, actor)] = stat
    total_count = sum(total_cnts)
    total_mean = int(np.mean(total_cnts))
    total_std = int(np.std(total_cnts))
    print(actor, (total_count, total_mean, total_std))

CHN (845144, 9390, 29827)
USA (1991526, 22128, 63176)
RUS (451413, 5072, 12037)
FRA (402149, 4468, 8343)
IND (86395, 970, 3324)


In [24]:
subregion_actor_stat

{('Southern Asia', 'CHN'): (375836, 41759, 75507),
 ('Northern Africa', 'CHN'): (27978, 4663, 7932),
 ('Middle Africa', 'CHN'): (3921, 560, 1054),
 ('Western Asia', 'CHN'): (50256, 3350, 2643),
 ('Western Africa', 'CHN'): (56666, 4358, 7536),
 ('Southern Africa', 'CHN'): (29525, 2952, 3785),
 ('South-eastern Asia', 'CHN'): (246133, 24613, 35020),
 ('Eastern Africa', 'CHN'): (19414, 1386, 1771),
 ('Central Asia', 'CHN'): (35415, 5902, 8074),
 ('Southern Asia', 'USA'): (797729, 88636, 164295),
 ('Northern Africa', 'USA'): (57526, 9587, 14543),
 ('Middle Africa', 'USA'): (6905, 986, 1471),
 ('Western Asia', 'USA'): (390120, 26008, 39875),
 ('Western Africa', 'USA'): (203391, 15645, 32860),
 ('Southern Africa', 'USA'): (65327, 6532, 9727),
 ('South-eastern Asia', 'USA'): (289600, 28960, 39695),
 ('Eastern Africa', 'USA'): (79861, 5704, 11567),
 ('Central Asia', 'USA'): (101067, 16844, 34222),
 ('Southern Asia', 'RUS'): (104043, 11560, 16619),
 ('Northern Africa', 'RUS'): (36625, 6104, 1064

In [25]:
total_cnts = []
for sr in subregions:
    if sr == 'Eastern Asia':
        continue
        
    cnts = []
    for actor in actors:
        for c in subregion_country[sr]:
            if (c == 'Russian Federation' and actor == 'RUS') or (c == 'India, Republic of' and actor == 'IND'):
                continue

            # fetch local report counts for this actor
            p = 'CountrySourceCounts'+ '/' + c + '/' + 'local_stats_2015-02-18_2021-03-08_' + actor + '.csv'
            if (os.path.isfile(p)):
                df_cnt = pd.read_csv(p)
                cnt = df_cnt['NumArticles'].values[0]
                cnts.append(cnt)
                total_cnts.append(cnt)

    total = sum(cnts)
    mean = int(np.mean(cnts))
    std = int(np.std(cnts))

    stat = (total, mean, std)

    subregion_actor_stat[(sr, 'ALL')] = stat
total_count = sum(total_cnts)
total_mean = int(np.mean(total_cnts))
total_std = int(np.std(total_cnts))
print('ALL', (total_count, total_mean, total_std))

ALL (3776627, 8429, 32860)


In [26]:
subregion_actor_stat

{('Southern Asia', 'CHN'): (375836, 41759, 75507),
 ('Northern Africa', 'CHN'): (27978, 4663, 7932),
 ('Middle Africa', 'CHN'): (3921, 560, 1054),
 ('Western Asia', 'CHN'): (50256, 3350, 2643),
 ('Western Africa', 'CHN'): (56666, 4358, 7536),
 ('Southern Africa', 'CHN'): (29525, 2952, 3785),
 ('South-eastern Asia', 'CHN'): (246133, 24613, 35020),
 ('Eastern Africa', 'CHN'): (19414, 1386, 1771),
 ('Central Asia', 'CHN'): (35415, 5902, 8074),
 ('Southern Asia', 'USA'): (797729, 88636, 164295),
 ('Northern Africa', 'USA'): (57526, 9587, 14543),
 ('Middle Africa', 'USA'): (6905, 986, 1471),
 ('Western Asia', 'USA'): (390120, 26008, 39875),
 ('Western Africa', 'USA'): (203391, 15645, 32860),
 ('Southern Africa', 'USA'): (65327, 6532, 9727),
 ('South-eastern Asia', 'USA'): (289600, 28960, 39695),
 ('Eastern Africa', 'USA'): (79861, 5704, 11567),
 ('Central Asia', 'USA'): (101067, 16844, 34222),
 ('Southern Asia', 'RUS'): (104043, 11560, 16619),
 ('Northern Africa', 'RUS'): (36625, 6104, 1064

In [27]:
subregion_actor_stat_dict = {
    'Southern Asia'     : [()   , ()   , ()   , ()   , ()   , ()   ],
    'Western Asia'      : [()   , ()   , ()   , ()   , ()   , ()   ],
    'Central Asia'      : [()   , ()   , ()   , ()   , ()   , ()   ],
    'South-eastern Asia': [()   , ()   , ()   , ()   , ()   , ()   ],
    'Northern Africa'   : [()   , ()   , ()   , ()   , ()   , ()   ],
    'Middle Africa'     : [()   , ()   , ()   , ()   , ()   , ()   ],
    'Western Africa'    : [()   , ()   , ()   , ()   , ()   , ()   ],
    'Southern Africa'   : [()   , ()   , ()   , ()   , ()   , ()   ],
    'Eastern Africa'    : [()   , ()   , ()   , ()   , ()   , ()   ],
    'Actor' : ['USA', 'IND', 'RUS', 'FRA', 'CHN', 'ALL']
}

for (subregion, actor), stat in subregion_actor_stat.items():
    subregion_actor_stat_dict[subregion][actor_pos[actor]] = stat

In [28]:
df_subregion = pd.DataFrame.from_dict(subregion_actor_stat_dict)
df_subregion.set_index('Actor')

,Southern Asia,Western Asia,Central Asia,South-eastern Asia,Northern Africa,Middle Africa,Western Africa,Southern Africa,Eastern Africa
Actor,,,,,,,,,
USA,"(797729, 88636, 164295)","(390120, 26008, 39875)","(101067, 16844, 34222)","(289600, 28960, 39695)","(57526, 9587, 14543)","(6905, 986, 1471)","(203391, 15645, 32860)","(65327, 6532, 9727)","(79861, 5704, 11567)"
IND,"(54618, 6827, 8776)","(11868, 791, 1247)","(1483, 247, 424)","(13051, 1305, 1536)","(574, 95, 166)","(53, 7, 15)","(1598, 122, 270)","(1011, 101, 165)","(2139, 152, 306)"
RUS,"(104043, 11560, 16619)","(225736, 15049, 21003)","(24486, 4897, 3906)","(38040, 3804, 4268)","(36625, 6104, 10641)","(847, 121, 169)","(11019, 847, 1513)","(7082, 708, 1017)","(3535, 252, 287)"
FRA,"(90323, 10035, 18427)","(78781, 5252, 5210)","(21293, 3548, 6762)","(38872, 3887, 4193)","(57882, 9647, 8722)","(13192, 1884, 2521)","(76844, 5911, 8620)","(13073, 1307, 2508)","(11889, 849, 713)"
CHN,"(375836, 41759, 75507)","(50256, 3350, 2643)","(35415, 5902, 8074)","(246133, 24613, 35020)","(27978, 4663, 7932)","(3921, 560, 1054)","(56666, 4358, 7536)","(29525, 2952, 3785)","(19414, 1386, 1771)"
ALL,"(1422549, 32330, 88330)","(756761, 10090, 22361)","(183744, 6336, 17333)","(625696, 12513, 26582)","(180585, 6019, 10262)","(24918, 711, 1548)","(349518, 5377, 16542)","(116018, 2320, 5347)","(116838, 1669, 5638)"


In [29]:
df_sub = pd.DataFrame.from_dict(subregion_actor_stat_dict)
df_sub.set_index('Actor')

,Southern Asia,Western Asia,Central Asia,South-eastern Asia,Northern Africa,Middle Africa,Western Africa,Southern Africa,Eastern Africa
Actor,,,,,,,,,
USA,"(797729, 88636, 164295)","(390120, 26008, 39875)","(101067, 16844, 34222)","(289600, 28960, 39695)","(57526, 9587, 14543)","(6905, 986, 1471)","(203391, 15645, 32860)","(65327, 6532, 9727)","(79861, 5704, 11567)"
IND,"(54618, 6827, 8776)","(11868, 791, 1247)","(1483, 247, 424)","(13051, 1305, 1536)","(574, 95, 166)","(53, 7, 15)","(1598, 122, 270)","(1011, 101, 165)","(2139, 152, 306)"
RUS,"(104043, 11560, 16619)","(225736, 15049, 21003)","(24486, 4897, 3906)","(38040, 3804, 4268)","(36625, 6104, 10641)","(847, 121, 169)","(11019, 847, 1513)","(7082, 708, 1017)","(3535, 252, 287)"
FRA,"(90323, 10035, 18427)","(78781, 5252, 5210)","(21293, 3548, 6762)","(38872, 3887, 4193)","(57882, 9647, 8722)","(13192, 1884, 2521)","(76844, 5911, 8620)","(13073, 1307, 2508)","(11889, 849, 713)"
CHN,"(375836, 41759, 75507)","(50256, 3350, 2643)","(35415, 5902, 8074)","(246133, 24613, 35020)","(27978, 4663, 7932)","(3921, 560, 1054)","(56666, 4358, 7536)","(29525, 2952, 3785)","(19414, 1386, 1771)"
ALL,"(1422549, 32330, 88330)","(756761, 10090, 22361)","(183744, 6336, 17333)","(625696, 12513, 26582)","(180585, 6019, 10262)","(24918, 711, 1548)","(349518, 5377, 16542)","(116018, 2320, 5347)","(116838, 1669, 5638)"


In [30]:
df_sub = pd.DataFrame.from_dict(subregion_actor_stat_dict)
df_sub

,Southern Asia,Western Asia,Central Asia,South-eastern Asia,Northern Africa,Middle Africa,Western Africa,Southern Africa,Eastern Africa,Actor
0,"(797729, 88636, 164295)","(390120, 26008, 39875)","(101067, 16844, 34222)","(289600, 28960, 39695)","(57526, 9587, 14543)","(6905, 986, 1471)","(203391, 15645, 32860)","(65327, 6532, 9727)","(79861, 5704, 11567)",USA
1,"(54618, 6827, 8776)","(11868, 791, 1247)","(1483, 247, 424)","(13051, 1305, 1536)","(574, 95, 166)","(53, 7, 15)","(1598, 122, 270)","(1011, 101, 165)","(2139, 152, 306)",IND
2,"(104043, 11560, 16619)","(225736, 15049, 21003)","(24486, 4897, 3906)","(38040, 3804, 4268)","(36625, 6104, 10641)","(847, 121, 169)","(11019, 847, 1513)","(7082, 708, 1017)","(3535, 252, 287)",RUS
3,"(90323, 10035, 18427)","(78781, 5252, 5210)","(21293, 3548, 6762)","(38872, 3887, 4193)","(57882, 9647, 8722)","(13192, 1884, 2521)","(76844, 5911, 8620)","(13073, 1307, 2508)","(11889, 849, 713)",FRA
4,"(375836, 41759, 75507)","(50256, 3350, 2643)","(35415, 5902, 8074)","(246133, 24613, 35020)","(27978, 4663, 7932)","(3921, 560, 1054)","(56666, 4358, 7536)","(29525, 2952, 3785)","(19414, 1386, 1771)",CHN
5,"(1422549, 32330, 88330)","(756761, 10090, 22361)","(183744, 6336, 17333)","(625696, 12513, 26582)","(180585, 6019, 10262)","(24918, 711, 1548)","(349518, 5377, 16542)","(116018, 2320, 5347)","(116838, 1669, 5638)",ALL


In [31]:
df_sub_cnt = pd.DataFrame([
    df_sub['Southern Asia'].apply(lambda r: r[0]), 
    df_sub['Northern Africa'].apply(lambda r: r[0]),
    df_sub['Middle Africa'].apply(lambda r: r[0]), 
    df_sub['Western Asia'].apply(lambda r: r[0]),
    df_sub['Western Africa'].apply(lambda r: r[0]), 
    df_sub['Southern Africa'].apply(lambda r: r[0]),
    df_sub['South-eastern Asia'].apply(lambda r: r[0]), 
    df_sub['Eastern Africa'].apply(lambda r: r[0]),
    df_sub['Central Asia'].apply(lambda r: r[0])
]).T
df_sub_cnt['Actor'] = ['USA', 'IND', 'RUS', 'FRA', 'CHN', 'ALL']
df_sub_cnt.set_index('Actor')
df_sub_cnt['Metric'] = 'count'
df_sub_cnt

,Southern Asia,Northern Africa,Middle Africa,Western Asia,Western Africa,Southern Africa,South-eastern Asia,Eastern Africa,Central Asia,Actor,Metric
0,797729,57526,6905,390120,203391,65327,289600,79861,101067,USA,count
1,54618,574,53,11868,1598,1011,13051,2139,1483,IND,count
2,104043,36625,847,225736,11019,7082,38040,3535,24486,RUS,count
3,90323,57882,13192,78781,76844,13073,38872,11889,21293,FRA,count
4,375836,27978,3921,50256,56666,29525,246133,19414,35415,CHN,count
5,1422549,180585,24918,756761,349518,116018,625696,116838,183744,ALL,count


In [32]:
df_sub_mean = pd.DataFrame([
    df_sub['Southern Asia'].apply(lambda r: r[1]), 
    df_sub['Northern Africa'].apply(lambda r: r[1]),
    df_sub['Middle Africa'].apply(lambda r: r[1]), 
    df_sub['Western Asia'].apply(lambda r: r[1]),
    df_sub['Western Africa'].apply(lambda r: r[1]), 
    df_sub['Southern Africa'].apply(lambda r: r[1]),
    df_sub['South-eastern Asia'].apply(lambda r: r[1]), 
    df_sub['Eastern Africa'].apply(lambda r: r[1]),
    df_sub['Central Asia'].apply(lambda r: r[1])
]).T
df_sub_mean['Actor'] = ['USA', 'IND', 'RUS', 'FRA', 'CHN', 'ALL']
df_sub_mean.set_index('Actor')
df_sub_mean['Metric'] = 'mean'
df_sub_mean

,Southern Asia,Northern Africa,Middle Africa,Western Asia,Western Africa,Southern Africa,South-eastern Asia,Eastern Africa,Central Asia,Actor,Metric
0,88636,9587,986,26008,15645,6532,28960,5704,16844,USA,mean
1,6827,95,7,791,122,101,1305,152,247,IND,mean
2,11560,6104,121,15049,847,708,3804,252,4897,RUS,mean
3,10035,9647,1884,5252,5911,1307,3887,849,3548,FRA,mean
4,41759,4663,560,3350,4358,2952,24613,1386,5902,CHN,mean
5,32330,6019,711,10090,5377,2320,12513,1669,6336,ALL,mean


In [33]:
df_sub_std = pd.DataFrame([
    df_sub['Southern Asia'].apply(lambda r: r[2]), 
    df_sub['Northern Africa'].apply(lambda r: r[2]),
    df_sub['Middle Africa'].apply(lambda r: r[2]), 
    df_sub['Western Asia'].apply(lambda r: r[2]),
    df_sub['Western Africa'].apply(lambda r: r[2]), 
    df_sub['Southern Africa'].apply(lambda r: r[2]),
    df_sub['South-eastern Asia'].apply(lambda r: r[2]), 
    df_sub['Eastern Africa'].apply(lambda r: r[2]),
    df_sub['Central Asia'].apply(lambda r: r[2])
]).T
df_sub_std['Actor'] = ['USA', 'IND', 'RUS', 'FRA', 'CHN', 'ALL']
df_sub_std.set_index('Actor')
df_sub_std['Metric'] = 'std'
df_sub_std

,Southern Asia,Northern Africa,Middle Africa,Western Asia,Western Africa,Southern Africa,South-eastern Asia,Eastern Africa,Central Asia,Actor,Metric
0,164295,14543,1471,39875,32860,9727,39695,11567,34222,USA,std
1,8776,166,15,1247,270,165,1536,306,424,IND,std
2,16619,10641,169,21003,1513,1017,4268,287,3906,RUS,std
3,18427,8722,2521,5210,8620,2508,4193,713,6762,FRA,std
4,75507,7932,1054,2643,7536,3785,35020,1771,8074,CHN,std
5,88330,10262,1548,22361,16542,5347,26582,5638,17333,ALL,std


In [34]:
df_sub = pd.concat([df_sub_cnt, df_sub_mean, df_sub_std]).sort_values('Actor')
df_sub = df_sub.set_index('Actor')
df_sub

,Southern Asia,Northern Africa,Middle Africa,Western Asia,Western Africa,Southern Africa,South-eastern Asia,Eastern Africa,Central Asia,Metric
Actor,,,,,,,,,,
ALL,88330,10262,1548,22361,16542,5347,26582,5638,17333,std
ALL,1422549,180585,24918,756761,349518,116018,625696,116838,183744,count
ALL,32330,6019,711,10090,5377,2320,12513,1669,6336,mean
CHN,375836,27978,3921,50256,56666,29525,246133,19414,35415,count
CHN,75507,7932,1054,2643,7536,3785,35020,1771,8074,std
CHN,41759,4663,560,3350,4358,2952,24613,1386,5902,mean
FRA,18427,8722,2521,5210,8620,2508,4193,713,6762,std
FRA,90323,57882,13192,78781,76844,13073,38872,11889,21293,count
FRA,10035,9647,1884,5252,5911,1307,3887,849,3548,mean


In [35]:
df_sub = df_sub.groupby([df_sub.index, 'Metric']).agg(float)
df_sub = df_sub.convert_dtypes()

In [36]:
df_sub

Southern Asia  Northern Africa  Middle Africa  Western Asia  \
Actor Metric                                                                
ALL   count         1422549           180585          24918        756761   
      mean            32330             6019            711         10090   
      std             88330            10262           1548         22361   
CHN   count          375836            27978           3921         50256   
      mean            41759             4663            560          3350   
      std             75507             7932           1054          2643   
FRA   count           90323            57882          13192         78781   
      mean            10035             9647           1884          5252   
      std             18427             8722           2521          5210   
IND   count           54618              574             53         11868   
      mean             6827               95              7           791   
      std              8776              166             15          1247   
RUS   count          104043            36625            847        225736   
      mean            11560             6104            121         15049   
      std             16619            10641            169         21003   
USA   count          797729            57526           6905        390120   
      mean            88636             9587            986         26008   
      std            164295            14543           1471         39875   

              Western Africa  Southern Africa  South-eastern Asia  \
Actor Metric                                                        
ALL   count           349518           116018              625696   
      mean              5377             2320               12513   
      std              16542             5347               26582   
CHN   count            56666            29525              246133   
      mean              4358             2952               24613   
      std               7536             3785               35020   
FRA   count            76844            13073               38872   
      mean              5911             1307                3887   
      std               8620             2508                4193   
IND   count             1598             1011               13051   
      mean               122              101                1305   
      std                270              165                1536   
RUS   count            11019             7082               38040   
      mean               847              708                3804   
      std               1513             1017                4268   
USA   count           203391            65327              289600   
      mean             15645             6532               28960   
      std              32860             9727               39695   

              Eastern Africa  Central Asia  
Actor Metric                                
ALL   count           116838        183744  
      mean              1669          6336  
      std               5638         17333  
CHN   count            19414         35415  
      mean              1386          5902  
      std               1771          8074  
FRA   count            11889         21293  
      mean               849          3548  
      std                713          6762  
IND   count             2139          1483  
      mean               152           247  
      std                306           424  
RUS   count             3535         24486  
      mean               252          4897  
      std                287          3906  
USA   count            79861        101067  
      mean              5704         16844  
      std              11567         34222

In [37]:
df_sub['Metric'] = df_sub.index.get_level_values(1)

In [38]:
df_sub

Southern Asia  Northern Africa  Middle Africa  Western Asia  \
Actor Metric                                                                
ALL   count         1422549           180585          24918        756761   
      mean            32330             6019            711         10090   
      std             88330            10262           1548         22361   
CHN   count          375836            27978           3921         50256   
      mean            41759             4663            560          3350   
      std             75507             7932           1054          2643   
FRA   count           90323            57882          13192         78781   
      mean            10035             9647           1884          5252   
      std             18427             8722           2521          5210   
IND   count           54618              574             53         11868   
      mean             6827               95              7           791   
      std              8776              166             15          1247   
RUS   count          104043            36625            847        225736   
      mean            11560             6104            121         15049   
      std             16619            10641            169         21003   
USA   count          797729            57526           6905        390120   
      mean            88636             9587            986         26008   
      std            164295            14543           1471         39875   

              Western Africa  Southern Africa  South-eastern Asia  \
Actor Metric                                                        
ALL   count           349518           116018              625696   
      mean              5377             2320               12513   
      std              16542             5347               26582   
CHN   count            56666            29525              246133   
      mean              4358             2952               24613   
      std               7536             3785               35020   
FRA   count            76844            13073               38872   
      mean              5911             1307                3887   
      std               8620             2508                4193   
IND   count             1598             1011               13051   
      mean               122              101                1305   
      std                270              165                1536   
RUS   count            11019             7082               38040   
      mean               847              708                3804   
      std               1513             1017                4268   
USA   count           203391            65327              289600   
      mean             15645             6532               28960   
      std              32860             9727               39695   

              Eastern Africa  Central Asia Metric  
Actor Metric                                       
ALL   count           116838        183744  count  
      mean              1669          6336   mean  
      std               5638         17333    std  
CHN   count            19414         35415  count  
      mean              1386          5902   mean  
      std               1771          8074    std  
FRA   count            11889         21293  count  
      mean               849          3548   mean  
      std                713          6762    std  
IND   count             2139          1483  count  
      mean               152           247   mean  
      std                306           424    std  
RUS   count             3535         24486  count  
      mean               252          4897   mean  
      std                287          3906    std  
USA   count            79861        101067  count  
      mean              5704         16844   mean  
      std              11567         34222    std

In [39]:
df_sub['Total'] = df_sub.apply(
    lambda x: x['Southern Asia'] + x['Northern Africa'] + x['Middle Africa'] + 
    x['Western Asia'] + x['Western Africa'] + x['Southern Africa'] + x['South-eastern Asia'] + 
    x['Eastern Africa'] + x['Central Asia'] if x['Metric'] == 'count' else '-', 
    axis=1
)

In [40]:
df_sub.drop(columns=['Metric'], inplace=True)

In [41]:
df_sub

Southern Asia  Northern Africa  Middle Africa  Western Asia  \
Actor Metric                                                                
ALL   count         1422549           180585          24918        756761   
      mean            32330             6019            711         10090   
      std             88330            10262           1548         22361   
CHN   count          375836            27978           3921         50256   
      mean            41759             4663            560          3350   
      std             75507             7932           1054          2643   
FRA   count           90323            57882          13192         78781   
      mean            10035             9647           1884          5252   
      std             18427             8722           2521          5210   
IND   count           54618              574             53         11868   
      mean             6827               95              7           791   
      std              8776              166             15          1247   
RUS   count          104043            36625            847        225736   
      mean            11560             6104            121         15049   
      std             16619            10641            169         21003   
USA   count          797729            57526           6905        390120   
      mean            88636             9587            986         26008   
      std            164295            14543           1471         39875   

              Western Africa  Southern Africa  South-eastern Asia  \
Actor Metric                                                        
ALL   count           349518           116018              625696   
      mean              5377             2320               12513   
      std              16542             5347               26582   
CHN   count            56666            29525              246133   
      mean              4358             2952               24613   
      std               7536             3785               35020   
FRA   count            76844            13073               38872   
      mean              5911             1307                3887   
      std               8620             2508                4193   
IND   count             1598             1011               13051   
      mean               122              101                1305   
      std                270              165                1536   
RUS   count            11019             7082               38040   
      mean               847              708                3804   
      std               1513             1017                4268   
USA   count           203391            65327              289600   
      mean             15645             6532               28960   
      std              32860             9727               39695   

              Eastern Africa  Central Asia    Total  
Actor Metric                                         
ALL   count           116838        183744  3776627  
      mean              1669          6336        -  
      std               5638         17333        -  
CHN   count            19414         35415   845144  
      mean              1386          5902        -  
      std               1771          8074        -  
FRA   count            11889         21293   402149  
      mean               849          3548        -  
      std                713          6762        -  
IND   count             2139          1483    86395  
      mean               152           247        -  
      std                306           424        -  
RUS   count             3535         24486   451413  
      mean               252          4897        -  
      std                287          3906        -  
USA   count            79861        101067  1991526  
      mean              5704         16844        -  
      std              11567         34222        -

In [42]:
df_sub['Southern Asia'] = df_sub.apply(lambda x: "{:,}".format(x['Southern Asia']), axis=1)
df_sub['Northern Africa'] = df_sub.apply(lambda x: "{:,}".format(x['Northern Africa']), axis=1)
df_sub['Middle Africa'] = df_sub.apply(lambda x: "{:,}".format(x['Middle Africa']), axis=1)
df_sub['Western Asia'] = df_sub.apply(lambda x: "{:,}".format(x['Western Asia']), axis=1)
df_sub['Western Africa'] = df_sub.apply(lambda x: "{:,}".format(x['Western Africa']), axis=1)
df_sub['Southern Africa'] = df_sub.apply(lambda x: "{:,}".format(x['Southern Africa']), axis=1)
df_sub['South-eastern Asia'] = df_sub.apply(lambda x: "{:,}".format(x['South-eastern Asia']), axis=1) 
df_sub['Eastern Africa'] = df_sub.apply(lambda x: "{:,}".format(x['Eastern Africa']), axis=1)
df_sub['Central Asia'] = df_sub.apply(lambda x: "{:,}".format(x['Central Asia']), axis=1)
df_sub['Total'] = df_sub.apply(lambda x: "{:,}".format(x['Total']) if x['Total'] != '-' else '-', axis=1)

In [43]:
df_sub

Southern Asia Northern Africa Middle Africa Western Asia  \
Actor Metric                                                            
ALL   count      1,422,549         180,585        24,918      756,761   
      mean          32,330           6,019           711       10,090   
      std           88,330          10,262         1,548       22,361   
CHN   count        375,836          27,978         3,921       50,256   
      mean          41,759           4,663           560        3,350   
      std           75,507           7,932         1,054        2,643   
FRA   count         90,323          57,882        13,192       78,781   
      mean          10,035           9,647         1,884        5,252   
      std           18,427           8,722         2,521        5,210   
IND   count         54,618             574            53       11,868   
      mean           6,827              95             7          791   
      std            8,776             166            15        1,247   
RUS   count        104,043          36,625           847      225,736   
      mean          11,560           6,104           121       15,049   
      std           16,619          10,641           169       21,003   
USA   count        797,729          57,526         6,905      390,120   
      mean          88,636           9,587           986       26,008   
      std          164,295          14,543         1,471       39,875   

             Western Africa Southern Africa South-eastern Asia Eastern Africa  \
Actor Metric                                                                    
ALL   count         349,518         116,018            625,696        116,838   
      mean            5,377           2,320             12,513          1,669   
      std            16,542           5,347             26,582          5,638   
CHN   count          56,666          29,525            246,133         19,414   
      mean            4,358           2,952             24,613          1,386   
      std             7,536           3,785             35,020          1,771   
FRA   count          76,844          13,073             38,872         11,889   
      mean            5,911           1,307              3,887            849   
      std             8,620           2,508              4,193            713   
IND   count           1,598           1,011             13,051          2,139   
      mean              122             101              1,305            152   
      std               270             165              1,536            306   
RUS   count          11,019           7,082             38,040          3,535   
      mean              847             708              3,804            252   
      std             1,513           1,017              4,268            287   
USA   count         203,391          65,327            289,600         79,861   
      mean           15,645           6,532             28,960          5,704   
      std            32,860           9,727             39,695         11,567   

             Central Asia      Total  
Actor Metric                          
ALL   count       183,744  3,776,627  
      mean          6,336          -  
      std          17,333          -  
CHN   count        35,415    845,144  
      mean          5,902          -  
      std           8,074          -  
FRA   count        21,293    402,149  
      mean          3,548          -  
      std           6,762          -  
IND   count         1,483     86,395  
      mean            247          -  
      std             424          -  
RUS   count        24,486    451,413  
      mean          4,897          -  
      std           3,906          -  
USA   count       101,067  1,991,526  
      mean         16,844          -  
      std          34,222          -

In [44]:
df_sub.to_csv('subregion_stats.csv')